# Aula 6: Introdução à Engenharia de Prompt

**Objetivo:** Implementar um classificador de emails simples usando um LLM (gpt-4o-mini) e os 4 componentes fundamentais da Engenharia de Prompt.

> **Conceitos-chave abordados**:
> - **Engenharia de Prompt**: A técnica de estruturar o texto de entrada (prompt) para obter a saída desejada de um Modelo de Linguagem.
> - **Componentes do Prompt**: O uso de tags (Persona, Task, Instructions, Output) para organizar e definir o comportamento do modelo.
> - **Classificação Zero-Shot**: A capacidade de um LLM classificar textos em categorias que ele não foi *explicitamente* treinado para, apenas seguindo instruções.

**Referências**

- [Documentação da API da OpenAI](https://platform.openai.com/docs/api-reference)
- [Guia de Engenharia de Prompt (OpenAI)](https://platform.openai.com/docs/guides/prompt-engineering)

## 📋 Fluxo do exercício

1.  **Configurar o ambiente** inserindo a `OPENAI_API_KEY`.
2.  **Analisar a Tarefa Prática** (Classificador de Email).
3.  **Completar os 4 componentes do prompt** (`aluno_persona`, `aluno_task`, `aluno_instructions`, `aluno_output`) na área indicada.
4.  **Executar o script de validação** para testar seu prompt contra 4 emails de exemplo e verificar se todos são classificados corretamente.

## ⚙️ 1. Ambiente e Dependências (API)

Preencha sua **API Key** da OpenAI. Você pode usar a variável de ambiente `OPENAI_API_KEY` ou inseri-la com segurança via `getpass`.

In [4]:
import os
import sys

os.environ["OPENROUTER_API_KEY"] = ""
# Entrada segura
try:
  from getpass import getpass
except ImportError:
  print("Aviso: Módulo getpass não encontrado, a chave ficará visível.")
  getpass = input

# 1) Tente ler da variável de ambiente (OpenRouter usa OPENROUTER_API_KEY)
api_key = os.environ.get("OPENROUTER_API_KEY", "").strip()

# 2) Se não houver, peça ao usuário (input seguro)
if not api_key and getpass is not None:
    api_key = getpass("Digite sua OPENROUTER_API_KEY: ").strip()
    os.environ["OPENROUTER_API_KEY"] = api_key
    print("Variável de ambiente OPENROUTER_API_KEY definida.")
else:
    print("Variável de ambiente OPENROUTER_API_KEY encontrada.")

# Se ainda assim não houver api, acuse um erro
if not api_key:
    raise RuntimeError("API key ausente. Defina OPENROUTER_API_KEY ou insira via getpass.")

# OpenRouter base URL
base_url = "https://openrouter.ai/api/v1"

# Compatibilidade com versões do SDK - configurando para OpenRouter
client = None
from openai import OpenAI
client = OpenAI(
    api_key=api_key,
    base_url=base_url,
    default_headers={
        "HTTP-Referer": "https://github.com/openrouter/openrouter",  # Opcional: para tracking
        "X-Title": "Teste OpenRouter",  # Opcional: nome da aplicação
    }
)

# Modelo disponível no OpenRouter (pode usar modelos de diferentes provedores)
# Exemplos: "openai/gpt-4o-mini", "anthropic/claude-3-haiku", "meta-llama/llama-3.1-8b-instruct"
MODEL = "openai/gpt-4o-mini"  # Usando o mesmo modelo através do OpenRouter
print(f"SDK detectado. Modelo: {MODEL} via OpenRouter")

# Utilitários para chamadas e exibição
def chat_complete(messages, temperature: float = 0.2, max_tokens: int = 300) -> str:
    """Chama o modelo com a lista de mensagens no formato [{'role': 'user'|'system'|'assistant', 'content': '...'}].
    Retorna apenas o conteúdo de texto da última resposta.

    Através do OpenRouter, você pode usar modelos de diferentes provedores:
    - OpenAI: "openai/gpt-4o-mini", "openai/gpt-4", "openai/gpt-3.5-turbo"
    - Anthropic: "anthropic/claude-3-haiku", "anthropic/claude-3-sonnet"
    - Meta: "meta-llama/llama-3.1-8b-instruct", "meta-llama/llama-3.1-70b-instruct"
    - Google: "google/gemini-pro", "google/gemini-flash"
    - Mistral: "mistralai/mistral-7b-instruct"

    Para trocar o modelo, altere a variável MODEL acima.
    """
    global client, MODEL
    try:
        resp = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content or ""
    except Exception as e:
        return f"[ERRO] {type(e).__name__}: {e}"

Digite sua OPENROUTER_API_KEY: ··········
Variável de ambiente OPENROUTER_API_KEY definida.
SDK detectado. Modelo: openai/gpt-4o-mini via OpenRouter


## ✉️ 2. Atividade Prática — **Classificador de Email** (Preencha e Execute)

Usando os **4 componentes**, crie um **prompt** para gerar um **sistema de análise de emails**.
- Saída deve ser **texto puro**, contendo apenas o motivo do email, que pode ser:
  - **Questão Técnica**
  - **Problema de Faturamento**
  - **Feedback de Produto**
  - **Desconhecido** (em caso de dúvida)

Após definir o conteúdo das tags,

> **SEÇÃO PARA O ALUNO PREENCHER:** edite as variáveis abaixo (**persona**, **task**, **instructions**, **output**). Depois, execute a célula para chamar o modelo.

In [5]:
from textwrap import dedent

# ======== ÁREA EDITÁVEL PELO ALUNO ========
aluno_persona = "você é um sistema de classificação de email"
aluno_task = "verificar o que tem no <context>"
aluno_instructions = """verificar se o email cabe em algum dos tipos
"Questão Técnica", "Problema de Faturamento", "Feedback de Produto", "Desconhecido"""
aluno_output = "Retorne apenas o nome exato da categoria, sem nenhuma outra palavra."

# ==========================================

# Exemplo de emails
emails = [
  ("""
  Assunto: Erro ao tentar fazer login

  Olá,

  Estou enfrentando um problema para acessar minha conta — toda vez que tento fazer login, aparece uma mensagem dizendo que meu usuário não existe. Já tentei redefinir a senha, mas nada mudou.

  Poderiam verificar, por favor?

  Obrigado,
  Rodrigo
  """, "Questão Técnica"),
  ("""
  Assunto: Cobrança duplicada na fatura de outubro

  Prezados,

  Notei que minha fatura deste mês apresenta dois débitos referentes ao mesmo serviço. Poderiam confirmar se houve cobrança duplicada e realizar o estorno?

  Atenciosamente,
  Fernanda
  """, "Problema de Faturamento"),
  ("""
  Assunto: Sugestão de melhoria no painel de relatórios

  Olá equipe,

  Tenho gostado bastante da nova interface, mas senti falta de uma opção para exportar relatórios em CSV diretamente do painel. Acredito que isso facilitaria muito o trabalho do time.

  Abraços,
  Marcelo
  """, "Feedback de Produto"),
  ("""
  Assunto: Parceria para evento acadêmico

  Boa tarde,

  Estou organizando um seminário sobre inovação e gostaria de saber se a empresa teria interesse em participar como apoiadora ou palestrante.

  Fico à disposição para conversar melhor.

  Cordialmente,
  Luana
  """, "Desconhecido"),
]

wrong_emails = []

for (index, (email, categoria_esperada)) in enumerate(emails, start=1):

    prompt_atividade = dedent(f"""
    <persona>{aluno_persona}</persona>
    <task>{aluno_task}</task>
    <instructions>{aluno_instructions}</instructions>
    <output>{aluno_output}</output>
    <context>
    Email:
    {email}
    </context>
    """)

    messages = [
        {"role": "user", "content": prompt_atividade}
    ]

    resposta_obtida = chat_complete(messages, temperature=0.1, max_tokens=50) # Reduzi max_tokens e temperature para classificação

    # Agora validamos se a resposta é uma das categorias esperadas (aproximado, pois o modelo pode variar a capitalização ou adicionar espaços)
    categorias_validas = ["Questão Técnica", "Problema de Faturamento", "Feedback de Produto", "Desconhecido"]

    # Flag de corretude
    resposta_correta = categoria_esperada.lower() == resposta_obtida.lower()
    if not resposta_correta:
        wrong_emails.append(index)

    # Defina um caractere de status de corretude
    result_char = "✅" if resposta_correta else "❌"

    print(f"{result_char} Classificação do Email {index}: (esperado / obtido) = ({categoria_esperada} / {resposta_obtida}).")

if len(wrong_emails) == 0:
    print("✅ Todos os emails foram classificados corretamente!")
else:
    print(f"❌ Os seguintes emails foram classificados erroneamente: {wrong_emails}")

✅ Classificação do Email 1: (esperado / obtido) = (Questão Técnica / Questão Técnica).
✅ Classificação do Email 2: (esperado / obtido) = (Problema de Faturamento / Problema de Faturamento).
✅ Classificação do Email 3: (esperado / obtido) = (Feedback de Produto / Feedback de Produto).
✅ Classificação do Email 4: (esperado / obtido) = (Desconhecido / Desconhecido).
✅ Todos os emails foram classificados corretamente!


## 🧪 Resumo da Tarefa do Aluno

O objetivo é criar um prompt de classificação de emails usando os 4 componentes-chave. Para isso, complete as quatro tarefas no script acima:

1.  **Definir Persona**: Na célula acima, defina `aluno_persona` para dizer ao modelo qual é o seu papel (ex: "Você é um sistema de classificação...").
2.  **Definir Tarefa (Task)**: Defina `aluno_task` para explicar o objetivo principal (ex: "Classificar o email em uma das 4 categorias...").
3.  **Definir Instruções (Instructions)**: Defina `aluno_instructions` para dar regras específicas (ex: "Responda apenas com a categoria", "Não inclua explicações").
4.  **Definir Formato de Saída (Output)**: Defina `aluno_output` para reforçar o formato exato da resposta (ex: "Apenas o nome da categoria...").

## 🧷 Dicas e Notas

- **Chave de API**: Se você receber um erro de autenticação, verifique se sua `OPENAI_API_KEY` foi inserida corretamente na primeira célula de código.
- **Modelo**: Estamos usando o `gpt-4o-mini`, que é rápido e eficiente para tarefas de classificação.
- **Temperatura**: Note que a função `chat_complete` usa `temperature=0.1`. Uma temperatura baixa é ideal para tarefas de classificação, pois queremos a resposta mais provável e menos criativa.
- **A Tag `<output>`**: A tag `<output>` é crucial. Ela ajuda a garantir que o modelo não adicione texto extra, como "Claro, a categoria é: Problema de Faturamento." Ele responderá apenas "Problema de Faturamento."